# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List, Tuple

In [2]:
def gini(x: np.ndarray) -> float:
    """
    Считает коэффициент Джини для массива меток x.
    """
    p = x / np.sum(x) if np.sum(x) > 0 else 0
    return np.sum(p * (1 - p))


def entropy(x: np.ndarray) -> float:
    """
    Считает энтропию для массива меток x.
    """
    p = x / np.sum(x) if np.sum(x) > 0 else 0
    return -np.sum(p * np.log(p))


def gain(left_y: np.ndarray, right_y: np.ndarray, criterion: Callable) -> float:
    """
    Считает информативность разбиения массива меток.

    Parameters
    ----------
    left_y : np.ndarray
        Левая часть разбиения.
    right_y : np.ndarray
        Правая часть разбиения.
    criterion : Callable
        Критерий разбиения.
    """
    node = left_y + right_y
    return np.sum(node) * criterion(node) - np.sum(left_y) * criterion(left_y) \
           - np.sum(right_y) * criterion(right_y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
class DecisionTreeLeaf:
    """

    Attributes
    ----------
    y : dict
        Словарь, отображающий метки в вероятность того, что объект, попавший в данный лист,
        принадлжит классу, соответствующиему метке
    """
    def __init__(self, y: np.ndarray, encodings: dict):
        self.y = encodings[np.argmax(y)]
        self.y_dict = self._convert_p(y, encodings)
    
    def _convert_p(self, y: np.ndarray, encodings: dict) -> dict:
        out = {}
        for i, v in enumerate(y):
            out[encodings[i]] = v
        return out

    def __str__(self):
        return f"[Leaf {self.y}]"


class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value.
    """
    def __init__(self, split_dim: int, split_value: float,
                 left: Union['DecisionTreeNode', DecisionTreeLeaf, np.ndarray],
                 right: Union['DecisionTreeNode', DecisionTreeLeaf, np.ndarray]):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

    def __str__(self):
        return f"[Node: left {self.left}, right {self.right}]"

In [4]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        mask = np.random.choice(np.arange(X.shape[0]), size=X.shape[0])
        mask_out = np.setdiff1d(np.arange(X.shape[0]), mask)
        self.classes = None
        self.encoding = None

        self.X = X[mask]
        self.X_out = X[mask_out]
        self.y = y[mask]
        self.y_encoded = self._encode_y(y, self.y)
        self.y_out = y[mask_out]
        self.criterion = gini if criterion == "gini" else entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_features = self._get_n_features()

        self.root = None

    def _get_n_features(self):
        if self.max_features == "auto":
            return int(np.sqrt(self.X.shape[1]))

    def _encode_y(self, y, y_masked):
        unique = np.unique(y)
        self.classes = len(unique)
        self.encoding = dict(enumerate(unique))

        # encoding y
        y_encoded = np.zeros_like(y_masked, dtype=int)
        reversed_encoding = {v: i for i, v in enumerate(unique)}
        for i, element in enumerate(y_masked):
            y_encoded[i] = reversed_encoding[element]
        return y_encoded

    def fit(self) -> NoReturn:
        self.root = self._build_tree(self.X, self.y_encoded, 0)

    def _build_tree(self, X: np.ndarray, y: np.ndarray, depth: int):
        if not len(X) or not self.min_samples_leaf:
            return DecisionTreeNode(split_dim=-1, split_value=-1,
                                    left=np.array([]), right=np.array([]))

        if self.max_depth is not None and depth == self.max_depth or len(np.unique(y)) == 1:
            return self._make_leaf(y)
        ###
        split_feature, split_value, left, right = self._get_separator(X, y)
        ###

        if len(left) < self.min_samples_leaf or len(right) < self.min_samples_leaf:
            return self._make_leaf(y)

        node = DecisionTreeNode(split_dim=split_feature, split_value=split_value,
                                left=left, right=right)

        node.left = self._build_tree(node.left[:, :-1], node.left[:, -1], depth + 1)
        node.right = self._build_tree(node.right[:, :-1], node.right[:, -1], depth + 1)

        return node

    def _get_separator(self, X: np.ndarray, y: np.ndarray) -> Tuple[int, float, np.ndarray, np.ndarray]:
        X_ = np.concatenate((X, y[:, None]), axis=1)
        n, f = X.shape
        features = np.random.choice(np.arange(f), replace=False, size=self.n_features)
        max_score = -np.inf
        split_feature, split_value = 0, 0
        left, right = None, None
        for feature in features:
            value = X_[0, feature]
            y_left = self._get_classes(X_[X_[:, feature] == value, -1])
            y_right = self._get_classes(X_[X_[:, feature] != value, -1])

            score = gain(y_left, y_right, self.criterion)

            if score > max_score:
                max_score = score
                split_feature = feature
                split_value = value
                left, right = X_[X_[:, feature] == split_value], X_[X_[:, feature] != split_value]

        return split_feature, split_value, left, right

    def _get_classes(self, y: np.ndarray) -> np.ndarray:
        out = np.zeros(self.classes)
        for element in y:
            out[int(element)] += 1
        return out

    def _make_leaf(self, y: np.ndarray) -> DecisionTreeLeaf:
        p = self._get_classes(y)
        p = p / np.sum(p)
        return DecisionTreeLeaf(p, self.encoding)

    def predict_proba(self, X: np.ndarray) -> List[Dict[Any, float]]:
        n, features = X.shape
        out = [{} for _ in range(n)]
        for i in range(n):
            root = self.root
            while not isinstance(root, DecisionTreeLeaf):
                feature, value = root.split_dim, root.split_value
                if X[i, feature] == value:
                    root = root.left
                else:
                    root = root.right
            # reached leaf
            out[i] = root.y_dict
        return out

    def predict(self, X: np.ndarray) -> list:
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [5]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None,
                 min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.trees = None
        self.y_type = None
        self.n_features = None

    def fit(self, X, y):
        self.y_type = y.dtype
        self.n_features = X.shape[1]
        self.trees = [DecisionTree(X, y, criterion=self.criterion, max_depth=self.max_depth,
                                   min_samples_leaf=self.min_samples_leaf, max_features=self.max_features)
                      for _ in range(self.n_estimators)]
        for tree in self.trees:
            tree.fit()

    def predict(self, X):
        n = X.shape[0]
        predictions = np.zeros((n, self.n_estimators), dtype=self.y_type)
        for i, tree in enumerate(self.trees):
            predictions[:, i] = np.array(tree.predict(X))
        output = np.zeros(n, dtype=self.y_type)
        for i in range(n):
            values, counts = np.unique(predictions[i], return_counts=True)
            output[i] = values[np.argmax(counts)]
        return output

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [6]:
def feature_importance(rfc):
    result = np.zeros((len(rfc.trees), rfc.n_features), dtype=np.float32)
    for i, tree in enumerate(rfc.trees):
        for feature in range(rfc.n_features):
            if len(tree.X_out) == 0:
                result[i, feature] = None
                continue
            err_oob = 1 - (np.mean(tree.predict(tree.X_out) == tree.y_out))
            X_shuf =  tree.X_out.copy()
            np.random.shuffle(X_shuf[:, feature])
            err_oob_j = 1 - (np.mean(tree.predict(X_shuf) == tree.y_out))
            result[i, feature] = err_oob_j - err_oob
    return np.nanmean(result, axis=0)


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [7]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [ 0.00059966  0.00069552  0.1682282   0.16412652  0.3157607  -0.00220471]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [10]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [54]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [86]:
%%time
rfc = RandomForestClassifier(n_estimators=20)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7414880201765448
Most important features:
1. ovsyanochan
2. 4ch
3. rhymes
4. mudakoff
5. styd.pozor
6. rapnewrap
7. dayvinchik
8. pravdashowtop
9. iwantyou
10. pixel_stickers
11. tumblr_vacuum
12. reflexia_our_feelings
13. leprum
14. i_d_t
15. pozor
16. ultrapir
17. bot_maxim
18. ne1party
19. thesmolny
20. ohhluul
CPU times: user 6min 10s, sys: 5.93 s, total: 6min 16s
Wall time: 6min 20s


#### Пол

In [87]:
%%time
rfc = RandomForestClassifier(n_estimators=20)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8474148802017655
Most important features:
1. 40kg
2. girlmeme
3. mudakoff
4. zerofat
5. 9o_6o_9o
6. modnailru
7. femalemem
8. 4ch
9. cook_good
10. be.women
11. be.beauty
12. woman.blog
13. igm
14. reflexia_our_feelings
15. rapnewrap
16. i_d_t
17. beauty
18. bon
19. sh.cook
20. thesmolny
CPU times: user 5min 30s, sys: 5.29 s, total: 5min 36s
Wall time: 5min 43s


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [8]:
X, y = synthetic_dataset(1000)
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = CatBoostClassifier(iterations=10,
                           learning_rate=0.01,
                           depth=5,
                           loss_function='MultiClass')
model.fit(X_train, y_train)
pred = model.predict(X_test)
print("Accuracy:", np.mean(pred.T[0] == y_test))
print(f"Importance: {np.round(model.feature_importances_, 2)}")

0:	learn: 1.0804341	total: 56.2ms	remaining: 505ms
1:	learn: 1.0619438	total: 58.3ms	remaining: 233ms
2:	learn: 1.0447217	total: 60.5ms	remaining: 141ms
3:	learn: 1.0279376	total: 62.6ms	remaining: 93.9ms
4:	learn: 1.0108608	total: 64.5ms	remaining: 64.5ms
5:	learn: 0.9942288	total: 66.3ms	remaining: 44.2ms
6:	learn: 0.9780233	total: 68ms	remaining: 29.1ms
7:	learn: 0.9628748	total: 70.2ms	remaining: 17.5ms
8:	learn: 0.9480850	total: 72.4ms	remaining: 8.04ms
9:	learn: 0.9330281	total: 74.1ms	remaining: 0us
Accuracy: 1.0
Importance: [ 0.    0.   28.82 28.63 42.55  0.  ]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Так же как и реализованный нами RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте. Выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [11]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [31]:
age = CatBoostClassifier(iterations=150,
                           learning_rate=0.1,
                           depth=10,
                           loss_function='MultiClass')
age.fit(X_train, y_age_train)
pred = age.predict(X_test)

In [30]:
print("Accuracy:", np.mean(pred.T[0] == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(age.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7604035308953342
Most important features:
1. ovsyanochan
2. dayvinchik
3. mudakoff
4. 4ch
5. styd.pozor
6. rhymes
7. rapnewrap
8. xfilm
9. leprum
10. bestad


#### Пол

In [33]:
sex = CatBoostClassifier(iterations=150,
                           learning_rate=0.01,
                           depth=10,
                           loss_function='MultiClass')
sex.fit(X_train, y_sex_train)
pred = sex.predict(X_test)

In [34]:
print("Accuracy:", np.mean(pred.T[0] == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(sex.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8600252206809584
Most important features:
1. 40kg
2. girlmeme
3. mudakoff
4. zerofat
5. cook_good
6. modnailru
7. i_d_t
8. thesmolny
9. femalemem
10. igm
